In [ ]:
import scipy.stats as stats
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# use yfinance library to download finance data from Yahoo! Finance
import yfinance as yf
from datetime import datetime, timedelta

# Setting beginning date and end date

In [ ]:
# Set the end date to September 30, 2023.
end_date = datetime(2023, 9, 30)

# Calculate the start date by subtracting 10 years from the end date.
beg_date = end_date - timedelta(days=3652)

print("Beginning Date is", beg_date)
print("End Date is", end_date)

Beginning Date is 2013-09-30 00:00:00
End Date is 2023-09-30 00:00:00


# Import Adj Close of Apple Inc. stock and Calculate Daily Return

In [ ]:
APPL = yf.download("AAPL",beg_date,end_date)["Adj Close"] #  adj closing prices
print("Latest value = ",APPL[-1])
APPL_dailyret = (APPL.diff()/APPL.shift(1))[1::] # calculate returns = (r_t - r_t-1)/r_t-1

[*********************100%%**********************]  1 of 1 completed
Latest value =  170.9847412109375


In [ ]:
APPL_dailyret

Date
2013-10-01    0.023513
2013-10-02    0.003279
2013-10-03   -0.012562
2013-10-04   -0.000786
2013-10-07    0.009772
                ...   
2023-09-25    0.007380
2023-09-26   -0.023398
2023-09-27   -0.008897
2023-09-28    0.001526
2023-09-29    0.003046
Name: Adj Close, Length: 2517, dtype: float64

# ข้อ 7: 1-day Sorting and Simulation VaR

## ข้อ 7.1: 1-day Sorting data VaR

In [ ]:
position = 1000000000
confidence_level = 0.99

#The VaR for the same stock based on sorting
APPL_dailyret_sort = np.sort(APPL_dailyret)

n = np.size(APPL_dailyret_sort)
print("n =",n)
t = int((1-confidence_level)*n)
print("Left tail =", t)
print("Sorted value at the left tail =", APPL_dailyret_sort[t-1])
VaR_sort = position * APPL_dailyret_sort[t-1]

print("Holding =", position, "Sorted 1-Day VaR =", round(VaR_sort,4), "tomorrow")

n = 2517
Left tail = 25
Sorted value at the left tail = -0.0480200333569158
Holding = 1000000000 Sorted 1-Day VaR = -48020033.3569 tomorrow


## ข้อ 7.2: 1-day Simulation VaR

In [ ]:
position = 1000000000
confidence_level = 0.99
n_simulation = 10000

# Calculate mean and sd for generate 10,000 returns with the same mean and standard deviation.
mean = np.mean(APPL_dailyret)
std = np.std(APPL_dailyret)

# Generate 10,000 returns with the same mean and standard deviation.
np.random.seed(999) # To ensure we get the same distribution everytime we run this script
APPL_dailyret_rand = np.random.normal(mean, std, n_simulation)

# Sort the simulated returns
APPL_dailyret_rand_sort = np.sort(APPL_dailyret_rand)
t = int(n_simulation * (1-confidence_level))
print("Left tail =", t)
print("Sorted value at the left tail =", APPL_dailyret_rand_sort[t-1])

# Calculate Simulation VaR
VaR_sim = position * (APPL_dailyret_rand_sort[t-1])
print("Holding =", position, "Simulation 1-day VaR =", round(VaR_sim, 4), "tomorrow")

Left tail = 100
Sorted value at the left tail = -0.040519947408046895
Holding = 1000000000 Simulation 1-day VaR = -40519947.408 tomorrow


# ข้อ 10: Normality test

In [ ]:
print('mean =', np.mean(APPL_dailyret))
print('std =',np.std(APPL_dailyret))
print('skewness=',stats.skew(APPL_dailyret))
print('kurtosis=',stats.kurtosis(APPL_dailyret))

mean = 0.0011319386994576978
std = 0.017905467042956355
skewness= -0.0312815006861748
kurtosis= 5.482150815814972


# ข้อ 11: 1-day Modified VaR

In [ ]:
position = 1000000000
confidence_level = 0.99

# Modified VaR: based on 4 moments
z = abs(stats.norm.ppf(1-confidence_level))
k = stats.kurtosis(APPL_dailyret)
s = stats.skew(APPL_dailyret)
t1 = (1/6.) * (z**2-1)* s
t2 = (1/24.) * (z**3-3*z) * k
t3 = (1/36.) * (2*z**3-5*z) * s**2
t = z + t1 + t2 - t3
mVaR = position * (mean - (t * std))
print("Holding =", round(position,2), "Modified VaR =", round(mVaR,2),"for 1 day ")

Holding = 1000000000 Modified VaR = -63052668.52 for 1 day 


# ข้อ 12: 30-days Simulation VaR

In [ ]:
APPL_30dayret = ((APPL.shift(30)-APPL)/APPL.shift(30))[30::] # calculate returns = (r_t - r_t-30)/r_t-30

In [ ]:
position = 1000000000
confidence_level = 0.99
n_simulation = 10000

# Calculate mean and sd for generate 10,000 returns with the same mean and standard deviation.
mean = np.mean(APPL_30dayret)
std = np.std(APPL_30dayret)

# Generate 10,000 returns with the same mean and standard deviation.
np.random.seed(999) # To ensure we get the same distribution everytime we run this script
APPL_30dayret_rand = np.random.normal(mean, std, n_simulation)

# Sort the simulated returns
APPL_30dayret_rand_sort = np.sort(APPL_30dayret_rand)
t = int(n_simulation * (1-confidence_level))
print("Left tail =", t)
print("Sorted value at the left tail =", APPL_30dayret_rand_sort[t-1])

# Calculate Simulation VaR
VaR_sim = position * (APPL_30dayret_rand_sort[t-1])
print("Holding =", position, "Simulation 30-day VaR =", round(VaR_sim, 4))

Left tail = 100
Sorted value at the left tail = -0.2594694664624283
Holding = 1000000000 Simulation 30-day VaR = -259469466.4624


# ข้อ 16: VaR for Portfolios

In [ ]:
confidence_level = 0.95 # confidence level
position = 1000000000 # total value of our portfolio

z = stats.norm.ppf(confidence_level)

# Step 4: get porfolio returns 30-days
portStd = 0.022
portMean = 0.011
VaR = position * (portMean - z*portStd)
print("Holding =",position, "VaR =", round(VaR,2), "tomorrow")

Holding = 1000000000 VaR = -25186779.79 tomorrow


# ข้อ 22 Expected Shortfall

In [ ]:
(((np.exp(-((1.65**2)/2)) / ((1-0.95)*np.sqrt(2*math.pi)))*0.022)+0.011)*1000000000

55996566.808150284